In [1]:
import wave
import requests
import time
import os
import numpy as np
import base64
import pyaudio
from pyaudio import PyAudio, paInt16
import webbrowser
from aip import AipSpeech # 导入api接口
from playsound import playsound	# 音频模块



In [2]:
framerate = 16000  # Sampling rate
num_samples = 2000  # Sampling checkpoints
channels = 1  # Sound channels
sampwidth = 2  # Sampling width in bytes (2 bytes)
FILEPATH = 'speech.wav'

base_url = "https://openapi.baidu.com/oauth/2.0/token?grant_type=client_credentials&client_id=%s&client_secret=%s"
APIKey = '1'  # Your own API Key, see https://ai.baidu.com/tech/speech for more detail
SecretKey = '' # Your own SecretKey, see https://ai.baidu.com/tech/speech for more detail

HOST = base_url % (APIKey, SecretKey)

client = AipSpeech('101110829', APIKey, SecretKey)


In [1]:
def play_sound(data):
    '''
    'per': 4  choose speaker，0: women, 1: man, 3：Emotional synthesis-Du Xiaoyao, 4: Emotional synthesis--Du Yaya
    '''
    result = client.synthesis(data, 'zh', 1, {
        'per': 3,
        'spd': 4,    # speed of speaking
        'vol': 6,   # volumn
        'pit':5
    })

    if not isinstance(result, dict):
        with open('D:/Code/JupyterNotebook/test.mp3', 'wb') as f:
            f.write(result)
        
        f.close()
    # play
    playsound("D:/Code/JupyterNotebook/test.mp3")
    


def getToken(host):
    res = requests.post(host)
    return res.json()['access_token']


def save_wave_file(filepath, data):
    wf = wave.open(filepath, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(sampwidth)
    wf.setframerate(framerate)
    wf.writeframes(b''.join(data))
    wf.close()

        
# 在上述代码中，我们通过`callback`函数来处理每次读取的音频帧。在`callback`函数中，
# 我们计算了音频帧的音量，并与阈值进行比较。当音量低于阈值时，我们停止录音，关闭音频流，并终止pyaudio。
# 你可以根据自己的需求调整阈值和其他参数，以适应不同的录音场景。
# 另外，以下是一些可能的阈值设置示例：
# 1. 静音录音：如果你只想录制有声音的部分，可以将阈值设置为一个较高的值，如2000。
# 2. 语音识别：如果你想录制语音用于语音识别，可以将阈值设置为一个适中的值，如1000。
# 3. 噪音检测：如果你想检测环境中的噪音水平，可以将阈值设置为一个较低的值，如500。
# 根据具体的应用场景和需求，你可以根据实际情况调整阈值和其他参数。

# In the above code, we use the ` callback ` function to process each audio frame read.
# In the 'callback' function, we calculated the volume of the audio frames and compared it with a threshold.
# When the volume is below the threshold, we stop recording, close the audio stream, and terminate pyaudio.
# You can adjust the threshold and other parameters according to your own needs to adapt to different recording scenarios.
# Additionally, the following are some possible examples of threshold settings:
# 1. Silent recording: If you only want to record the part with sound, you can set the threshold to a higher value, such as 2000.
# 2. Speech recognition: If you want to record speech for speech recognition, you can set the threshold to a moderate value, such as 1000.
# 3. Noise detection: If you want to detect the level of noise in the environment, you can set the threshold to a lower value, such as 500.
#According to specific application scenarios and requirements, you can adjust the threshold and other parameters based on the actual situation.


def my_record():
    CHUNK = 1024  # Size of each audio frame read
    FORMAT = paInt16  # Audio format
    CHANNELS = 1  # Number of channels
    RATE = 16000  # Sampling rate
    THRESHOLD = 1000  # Volume threshold
    delayTime=1.3  # Automatically stop after 1.3 seconds of low volume    
    
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)
    # snowboydecoder.play_audio_file()
#     print("Start")

    frames = []
    flag = False      # Start recording node
    stat = True       # Determine whether to continue recording
    stat2 = False     # Determine if the sound is low

    tempnum = 0   #tempnum、tempnum2、tempnum3为时间
    tempnum2 = 0

    while stat:
        data = stream.read(CHUNK,exception_on_overflow = False)
        frames.append(data)
        audio_data = np.frombuffer(data, dtype=np.short)
        temp = np.max(audio_data)
        if temp > THRESHOLD and flag==False:
            flag =True
#             print("Start record")
            tempnum2=tempnum
        if flag:

            if(temp < THRESHOLD and stat2==False):
                stat2 = True
                tempnum2 = tempnum
#                 print("low volumn, recording the current point.")
            if(temp > THRESHOLD):
                stat2 =False
                tempnum2 = tempnum
                # 刷新
            if(tempnum > tempnum2 + delayTime*15 and stat2==True):
#                 print("Start checking if it's still quiet after an interval of %.2lfs." % delayTime)
                if(stat2 and temp < THRESHOLD):
                    stat = False
                    #还是小声，则stat=True
#                     print("Low volumn")
                else:
                    stat2 = False
#                     print("Large volumn")


#         print(str(temp)  +  "      " +  str(tempnum))
        tempnum = tempnum + 1
        if tempnum > 150:  # Exit directly upon timeout
            stat = False
#     print("Stop record")
    p.terminate()
#     print(data)
    save_wave_file(FILEPATH, frames)
    stream.close()


def get_audio(file):
    with open(file, 'rb') as f:
        data = f.read()
    return data


def speech2text(speech_data, token, dev_pid=1537):
    FORMAT = 'wav'
    RATE = '16000'
    CHANNEL = 1
    CUID = '*******'
    SPEECH = base64.b64encode(speech_data).decode('utf-8')

    data = {
        'format': FORMAT,
        'rate': RATE,
        'channel': CHANNEL,
        'cuid': CUID,
        'len': len(speech_data),
        'speech': SPEECH,
        'token': token,
        'dev_pid':dev_pid
    }
    url = 'https://vop.baidu.com/server_api'
    headers = {'Content-Type': 'application/json'}
    # r=requests.post(url,data=json.dumps(data),headers=headers)
    print('Identifying...')
    r = requests.post(url, json=data, headers=headers)
    Result = r.json()
    if 'result' in Result:
        return Result['result'][0]
    else:
        return Result


In [4]:
Q_list = [
    '首先，请提供您的姓名、年龄、性别以及手机联系方式',
    '好的，请问您是什么时候开始发现有肺结节的？做过哪些检查？',
    '您是否记得结节位于主要在肺的哪个部位？你过去随访过程中结节是否有变化，是否曾就医治疗？',
    '您过去有没有患过高血压、糖尿病这一类的慢性疾病，或者对什么药物或物质有过敏反应？是否接受过大型手术？',
    '您平时是否抽烟或者饮酒？具体频率如何？另外，您近期饮食、睡眠如何，是否有不舒服的情况？',
    '好的，感谢您的配合！最后，请问您的爱人、子女以及亲戚朋友中有没有类似情况的健康问题？',
    '好的，我已经大概搜集好您的情况，您是否有需要补充的？请您告诉我。'
]

# devpid = input('1536：普通话(简单英文),1537:普通话(有标点),1737:英语,1637:粤语,1837:四川话\n')
devpid = 1537
All_Result = []

play_sound('您好，我是筛查机器人，负责采集您的一些信息，我将就您肺结节的情况向您咨询一些问题，请您配合我。')
play_sound('下面，我们开始。')


for item in Q_list:
    print(item)
    play_sound(item)
    print('请回答')
    my_record()
    TOKEN = getToken(HOST)
    speech = get_audio(FILEPATH)
    result = speech2text(speech, TOKEN, int(devpid))
    print(result)
    All_Result.append('%s: %s' % (item, result))
play_sound('感谢您的配合，我已将相关信息整理并且发送给您的负责医生，请您进一步至医院就诊。')

首先，请提供您的姓名、年龄、性别以及手机联系方式
请回答
正在识别...
我叫小明，我是男的，年龄24岁，我的手机号是13813645023。
好的，请问您是什么时候开始发现有肺结节的？做过哪些检查？
请回答
正在识别...
我是去年三月体检查了个胸部CT好像是报告说有个肺结节，然后医生就让我每半年。
您是否记得结节位于主要在肺的哪个部位？你过去随访过程中结节是否有变化，是否曾就医治疗？
请回答
正在识别...
我的结节好像是位于右上肺，我之前查了几次都没什么变化。
您过去有没有患过高血压、糖尿病这一类的慢性疾病，或者对什么药物或物质有过敏反应？是否接受过大型手术？
请回答
正在识别...
我平时身体挺好的，有高血压，在吃药，但是我没有糖尿病，好像也没有什么过敏手术，我也没有做。
您平时是否抽烟或者饮酒？具体频率如何？另外，您近期饮食、睡眠如何，是否有不舒服的情况？
请回答
正在识别...
我不喝酒，但是我抽烟，一包能抽2到3天，抽了八年了然后最近我吃饭睡觉都挺好的。
好的，感谢您的配合！最后，请问您的爱人、子女以及亲戚朋友中有没有类似情况的健康问题？
请回答
正在识别...
对我姐姐之前也查出来有肺结节，其他人没有都挺健康的。
好的，我已经大概搜集好您的情况，您是否有需要补充的？请您告诉我。
请回答
正在识别...
我没啥要补充的了。
